# sEMG Hand State Classification
This notebook reproduces the full workflow: data loading, preprocessing, windowing, and KNN classification.

## 1. Load Required Libraries and Dataset
This cell imports necessary libraries and loads the raw dataset from a CSV file.

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Load your raw dataset (replace with your actual path if needed)
# For now, this path is a placeholder; update before running
raw_path = "jared.csv"
try:
    data = pd.read_csv(raw_path)
    data.head()
except FileNotFoundError:
    print("Please upload jared.csv or update path.")

## 2. Split Data by Muscle State
This cell defines a function to split the dataset into closed and open fist states.

In [ ]:
# Split by muscle_state
def split_data(df):
    closedFist = df[df['muscle_state'] == 0]
    openFist = df[df['muscle_state'] == 1]
    return closedFist, openFist

try:
    closedFist, openFist = split_data(data)
    print("Split complete:", len(closedFist), "closed,", len(openFist), "open")
except:
    pass

## 3. Windowing Data
This cell defines a function to window the data every 5 rows and prepares the mutated dataset.

In [ ]:
# Windowing every 5 rows
def mutate_data(df):
    mutated_rows = []
    for i in range(0, len(df), 5):
        chunk = df.iloc[i:i+5]
        if len(chunk) == 5:
            readings = []
            for v in chunk['analog_reading'].tolist():
                if pd.isna(v):
                    readings.append(None)
                else:
                    try:
                        readings.append(int(v))
                    except:
                        readings.append(int(float(v)))
            mutated_rows.append({
                'analog_reading': readings,
                'muscle_state': int(chunk['muscle_state'].iloc[0])
            })
    return pd.DataFrame(mutated_rows)

try:
    open_mut = mutate_data(openFist)
    closed_mut = mutate_data(closedFist)
    combined = pd.concat([open_mut, closed_mut], ignore_index=True)
    combined.head()
except:
    pass

## 4. Parse Windowed Data and Train KNN Classifier
This cell parses the windowed analog readings, prepares features and labels, splits the data, trains a KNN classifier, and prints the accuracy.

In [ ]:
def parse_analog(cell):
    if pd.isna(cell):
        return None
    if isinstance(cell, list):
        return [float(x) if x is not None else np.nan for x in cell]
    if isinstance(cell, str):
        try:
            val = ast.literal_eval(cell)
            return [float(x) if x is not None else np.nan for x in val]
        except:
            return None
    return None

try:
    parsed = combined['analog_reading'].apply(parse_analog)
    valid = parsed.dropna()
    lengths = valid.map(len)
    common_len = lengths.mode().iloc[0]
    valid = valid[lengths == common_len]

    y = combined.loc[valid.index, 'muscle_state']
    mask = y.notna()
    valid = valid[mask]
    y = y[mask]

    X = np.vstack(valid.values).astype(float)
    y = y.astype(int).values

    trainX, testX, trainY, testY = train_test_split(X, y, random_state=42)
    model = KNeighborsClassifier()
    model.fit(trainX, trainY)

    print("Accuracy:", model.score(testX, testY))
except Exception as e:
    print("Model failed:", e)